In [1]:
# loading in packages and data queried from CBA database
import pandas as pd
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

df = pd.read_csv('~/projects/NFL Five Factors/plays_in_drives.csv', header=0)
turnovers = pd.read_csv('~/projects/NFL Five Factors/turnovers.csv', header=0)
position = pd.read_csv('~/projects/NFL Five Factors/field_position.csv', header=0)
home = pd.read_csv('~/projects/NFL Five Factors/home.csv', header=0)
visiting = pd.read_csv('~/projects/NFL Five Factors/visiting.csv', header=0)

In [2]:
# cleaning
plays = df.fillna(method='ffill')
plays['reverse_play_number_in_drive'] = plays.sort_values(['play_number_in_drive'], ascending = [False]).groupby(['game_id','team','drive_id']).cumcount() + 1
plays['drive_id'].astype('int32')

0         80885
1         80885
2         80885
3         80886
4         80886
5         80886
6         80886
7         80886
8         80886
9         80887
10        80887
11        80887
12        80887
13        80887
14        80887
15        80887
16        80887
17        80887
18        80887
19        80888
20        80888
21        80888
22        80888
23        80888
24        80888
25        80888
26        80889
27        80889
28        80889
29        80889
          ...  
106362    99641
106363    99641
106364    99641
106365    99641
106366    99641
106367    99641
106368    99642
106369    99642
106370    99642
106371    99643
106372    99643
106373    99643
106374    99644
106375    99644
106376    99644
106377    99645
106378    99645
106379    99645
106380    99646
106381    99646
106382    99646
106383    99647
106384    99647
106385    99647
106386    99648
106387    99648
106388    99648
106389    99649
106390    99649
106391    99649
Name: drive_id, Length: 

In [3]:
# this query finds the number of points scored in each drive
q = """
SELECT points_scored,
       drive_id
FROM plays
WHERE reverse_play_number_in_drive = 1
"""

drivepoints = pysqldf(q)

In [4]:
# this query finds the number of points scored per scoring opportunity
q2 = """
SELECT game_id,
       team,
       AVG(points_scored) AS points_per_trip_inside_forty,
       season
FROM (
        SELECT DISTINCT
               p.game_id,
               p.team,
               p.drive_id,
               p.season,
               CASE WHEN p.yardline >= 60 THEN 1
                    ELSE 0
                    END AS scoring_zone,
               p.points_scored

        FROM plays p
        JOIN
            (SELECT game_id,
                    drive_id,
                    team,
                    points_scored
             FROM plays
             WHERE reverse_play_number_in_drive = 1) a ON a.game_id = p.game_id
                                                      AND a.drive_id = p.drive_id
                                                      AND a.team = p.team
        WHERE scoring_zone = 1)

GROUP BY game_id,
         team,
         season
"""

ppo = pysqldf(q2)

In [5]:
# this query finds the expected points of a drive by down, distance, and yardline
q3 = """
SELECT p.down,
       p.distance,
       p.yardline,
       AVG(d.points_scored) AS exp_points
FROM plays p
JOIN drivepoints d ON d.drive_id = p.drive_id
GROUP BY p.down,
         p.distance, 
         p.yardline
"""

lookup = pysqldf(q3)

In [6]:
# this query finds the expected points for each play

q4 = """
SELECT p.team,
       p.game_id,
       p.play_number_in_drive,
       p.drive_id,
       p.points_scored,
       l.exp_points
FROM plays p
JOIN lookup l ON l.down = p.down
             AND l.distance = p.distance
             AND l.yardline = p.yardline

"""

expected = pysqldf(q4)

In [7]:
# this query success rate by team and game

q5 = """
SELECT s.team,
       s.game_id,
       AVG(s.success) AS success_rate
FROM
(SELECT e.team,
       e.game_id,
       e.play_number_in_drive,
       e.drive_id,
       e.exp_points,
       e2.exp_points AS next_exp_points,
       CASE WHEN e.points_scored > 0 THEN 1
            WHEN e.exp_points <= e2.exp_points THEN 1
            ELSE 0
       END AS success
FROM expected e
LEFT JOIN expected e2 ON e.drive_id = e2.drive_id
                  AND e.play_number_in_drive = (e2.play_number_in_drive - 1)) s
GROUP BY s.team,
         s.game_id
"""

success = pysqldf(q5)

In [8]:
# this query finds average gained expected points on successful plays

q6 = """
SELECT s.team,
       s.game_id,
       AVG((CASE WHEN s.points_scored > 0 THEN points_scored ELSE s.next_exp_points END) - s.exp_points) AS explosiveness
FROM
(SELECT e.team,
       e.game_id,
       e.play_number_in_drive,
       e.drive_id,
       e.points_scored,
       e.exp_points,
       e2.exp_points AS next_exp_points,
       CASE WHEN e.points_scored > 0 THEN 1
            WHEN e.exp_points <= e2.exp_points THEN 1
            ELSE 0
       END AS success
FROM expected e
LEFT JOIN expected e2 ON e.drive_id = e2.drive_id
                  AND e.play_number_in_drive = (e2.play_number_in_drive - 1)) s
WHERE s.success = 1
GROUP BY s.team,
         s.game_id
"""

explode = pysqldf(q6)

In [9]:
# this query brings all of the data together, ready for analysis

q7 = """
SELECT s.team,
       s.game_id,
       s.success_rate,
       e.explosiveness,
       ppo.points_per_trip_inside_forty,
       t.turnover_diff,
       p.average_starting_field_position,
       COALESCE(h.win, v.win) AS win
       
FROM success s
JOIN ppo ON ppo.team = s.team AND ppo.game_id = s.game_id
JOIN explode e ON e.team = s.team AND e.game_id = s.game_id
JOIN turnovers t ON t.team = s.team AND t.game_id = s.game_id
JOIN position p ON p.team = s.team AND p.game_id = s.game_id
LEFT JOIN home h ON h.home_team = s.team AND h.game_id = s.game_id
LEFT JOIN visiting v ON v.visiting_team = s.team AND v.game_id = s.game_id 

WHERE COALESCE(h.win, v.win) IS NOT NULL
"""

data = pysqldf(q7)

data

,team,game_id,success_rate,explosiveness,points_per_trip_inside_forty,turnover_diff,average_starting_field_position,win
0,ARI,3468,0.520548,0.692724,2.125000,0,19.0909,0
1,ARI,3490,0.418182,0.951266,2.333333,-1,28.5455,0
2,ARI,3511,0.406250,0.842687,1.625000,0,35.5714,1
3,ARI,3545,0.492537,1.102752,3.250000,-2,20.5333,0
4,ARI,3609,0.506667,1.049977,2.500000,2,23.3571,1
5,ARI,3638,0.591549,0.824699,2.333333,-3,18.7692,0
6,ARI,3674,0.534247,0.839556,2.500000,2,31.4167,1
7,ARI,3690,0.467532,0.914366,1.416667,-2,31.8667,1
8,ARI,3745,0.507692,0.581331,1.900000,3,33.0000,1
9,ARI,3794,0.305085,1.122533,2.166667,2,29.0000,0


In [10]:
data.describe()

,game_id,success_rate,explosiveness,points_per_trip_inside_forty,turnover_diff,average_starting_field_position,win
count,808.000000,808.000000,808.000000,808.000000,808.000000,808.000000,808.000000
mean,3851.237624,0.477425,0.955763,2.161157,0.017327,27.161563,0.429455
std,234.253748,0.063212,0.168097,0.781114,1.901463,5.703658,0.495305
min,3456.000000,0.285714,0.511849,-1.400000,-5.000000,14.666700,0.000000
25%,3648.750000,0.437280,0.840619,1.710714,-1.000000,23.166700,0.000000
50%,3851.500000,0.478873,0.938523,2.166667,0.000000,26.400000,0.000000
75%,4054.250000,0.520548,1.055926,2.625000,1.000000,30.339250,1.000000
max,4256.000000,0.644737,1.739314,7.000000,6.000000,49.166700,1.000000
